# Setup

In [ ]:
# Standard includes
%matplotlib inline
# %matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# Style setup
import seaborn as sns

sns.set_palette("muted")
sns.set_color_codes()
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in", "ytick.direction": "in"})
sns.set_style({"axes.grid": "True", "grid.color": "0.95"})

plt.rcParams["figure.figsize"] = [6, 6]
plt.rcParams["figure.dpi"] = 100


def darken_color(color, p):
    return (color[0] * p, color[1] * p, color[2] * p)


import matplotlib as mpl

colors = sns.color_palette("muted") + [(0.1, 0.1, 0.1)]
for code, color in zip(["bd", "gd", "rd", "md", "yd", "cd", "kd"], colors):
    rgb = mpl.colors.colorConverter.to_rgb(darken_color(color, 0.8))
    mpl.colors.colorConverter.colors[code] = rgb
    mpl.colors.colorConverter.cache[code] = rgb

blue = (114 / 256, 147 / 256, 203 / 256)
orange = (225 / 256, 151 / 256, 76 / 256)
green = (132 / 256, 186 / 256, 91 / 256)
red = (211 / 256, 94 / 256, 96 / 256)
grey = (128 / 256, 133 / 256, 133 / 256)
violet = (144 / 256, 103 / 256, 167 / 256)
brown = (171 / 256, 104 / 256, 87 / 256)
yellow = (204 / 256, 194 / 256, 16 / 256)

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

PLOT_DIR = "plots/"
FORMAT = ".png"

# Plots

In [ ]:
df_old = pd.read_csv("../data/data_old.csv", header=None, index_col=0)

In [ ]:
df_new = pd.read_csv(
    "~/Dropbox/1_Projects/ISS/data/DATA_20220224_tail.CSV", header=None, index_col=0
)

In [ ]:
df_220301 = pd.read_csv("../data/DATA_220301.csv", header=None, index_col=0)
df_220301_47k = pd.read_csv("../data/DATA_220301_47k.csv", header=None, index_col=0)
df_220301_47k_2 = pd.read_csv("../data/DATA_220301_47k_2.CSV", header=None, index_col=0)
df_220301_47k_ferro = pd.read_csv("../data/DATA_220301_47k_ferro.CSV", header=None, index_col=0)

In [ ]:
def overlay_plot(data, savefig=None):
    i = 0
    while i < 40 and i < len(data):
        if data.iloc[i].var() > 0.01:
            data.iloc[i].plot(
                xlabel="Time [ms]",
                ylabel="Voltage [V]",
            )
        i+=1
    if savefig:
        plt.savefig(savefig)

In [ ]:
overlay_plot(df_old, "before.png")

In [ ]:
overlay_plot(df_new, "after.png")

In [ ]:
overlay_plot(df_220301_47k)

In [ ]:
overlay_plot(df_220301_47k.iloc[:-2:2])

In [ ]:
overlay_plot(df_220301_47k.iloc[1:-2:2])

In [ ]:
overlay_plot(df_220301_47k_2.iloc[:-2:2])

In [ ]:
overlay_plot(df_220301_47k_2.iloc[1:-2:2].query("@df[5] > 2.3 & @df[6] > 2.3"))

In [ ]:
df = df_220301_47k_ferro

In [ ]:
overlay_plot(df.iloc[:-2:2])

In [ ]:
overlay_plot(df.iloc[1:-2:2].query("@df[5] < 2.8"))